####  Ngozi Ihemelandu

# CS534 Homework 3

<font color='red'>Good job</font> 
Put your homework in the directory with your name. Please mentionin this file the names of any students with whom you collaborated. If you didn't collaborate with anyone, mark your collaborators as "None." Remember, your goal is to communicate. Full credit will be given only to correct solutions which are described clearly. Convoluted and obtuse descriptions will receive low marks. To complete your homework, you may ONLY consult the following material:

lecture slides course notes you or others took during lecture. the required text (CLRS) websites that may clarify the concepts covered in the material but do not in any way provide complete solutions to the problems. Deadline 2/20/2019

Please provide an answer to the following question:

# Question 1 (15 pts)
<font color='red'> the procedure to find the best transformation as a part of the training procedure is not provided -2 </font>

To classify iris dataset (Iris-Versicolor vs. others) in the best way, you have to create an algorithm able to determine (with the k-fold cross validation) what is the best space transformation (among rbf_kernel, polynomial features, and polynomial kernel) and its hyperparameters. Each transformation has its own parameters rbf_kernel->gamma, polynomial-features-> degree, polynomial_kernel ->(gamma, degree). The performance has to be tested of the entire algorithm with another K-fold cross-validation (please use then trainset, validationset, and testset).

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import rbf_kernel, polynomial_kernel
from sklearn import svm
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support,accuracy_score,classification_report

In [2]:
#load the iris data
filename = "iris.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
data_iris = pd.read_csv(filename, names=names)
data_iris.head()

sepal-length  sepal-width  petal-length  petal-width        class
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa

In [3]:
#Put the independent and dependent variables into numpy arrays
indp_var = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width']
X = data_iris[indp_var].values
Y = data_iris["class"]=='Iris-versicolor'

#classify iris dataset (Iris-Versicolor vs. others)
Y = Y.values.astype(int)

In [4]:
# possible values of parameters to optimize over for space transformation 
# (rbf_kernel, polynomial features, and polynomial kernel)

rbf_param = {'gamma': [0.0001, 0.001, 0.01, 0.1, 0.5, 0.9, 1,10, 100, 1000]}
polyKernel_param = {'gamma': [0.001, 0.01, 0.1, 0.5, 0.9, 1, 10], 'degree': [1,2,3,4,5]}
polyFeatures_param = {'degree': [1,2,3,4,5]}

param_grid = dict(rbf=rbf_param, poly_Kernel=polyKernel_param, poly_Features=polyFeatures_param)

In [110]:
# Hyper-parameter Selection
def model_selection(X_train, y_train, Parameters, n_splits, kFunction):
    
    n_splits = n_splits
    folds_scores = {} 
    
    skf = StratifiedKFold(n_splits=n_splits, random_state=123, shuffle=True) #Stratified Cross Validation 
    cvFolds = list(skf.split(X_train, y_train))
    for count, fold in enumerate(cvFolds):
        
        train_index = fold[0]
        test_index = fold[1]
        
        cvX_train, cvX_test = X_train[train_index], X_train[test_index]
        cvy_train, cvy_test = y_train[train_index], y_train[test_index]

        parameters_scores = {} # dict to hold score for every choice of parameter value
        foldi = "fold"+str(count+1)
       
        #Learn a model on cvX_train for every choice of hyper-parameter value pi(Parameter)
        for Parameter in Parameters:
                        
            if kFunction == "rbf":    # rbf_kernel
                
                # Get the (N x N) Gram matrix(Kernel matrix) for every data point pair using the rbf kernel function
                # where N is the number of data points in the training data set
                cvXK_train = rbf_kernel(cvX_train, gamma=Parameter)
                
                # Get the (M x N) Gram matrix(Kernel matrix) for every data point pair using the Polynomial Kernel function
                # where M is the number of data points in the validation data set and N points in train set             
                cvXK_test = rbf_kernel(cvX_test,cvX_train, gamma=Parameter)
                
                # Set kernel='precomputed'and pass the Gram matrix(kernel matrix) [cvXK_train] instead of 
                # the train set [cvX_train] in the fit method                 
                kernel_type = 'precomputed'
                                
            if kFunction == "poly_Kernel":    # Polynomial Kernel
                
                gamma = Parameter[0]
                degree = Parameter[1]
                
                # Get the (N x N) Gram matrix(Kernel matrix) for every data point pair using the Polynomial Kernel function
                # where N is the number of data points in the training data set
                cvXK_train = polynomial_kernel(cvX_train, gamma=gamma, degree=degree)

                # Get the (M x N) Gram matrix(Kernel matrix) for every data point pair using the Polynomial Kernel function
                # where M is the number of data points in the validation data set and N the number of points in the train set
                cvXK_test = polynomial_kernel(cvX_test,cvX_train, gamma=gamma, degree=degree)
 
                # Set kernel='precomputed'and pass the Gram matrix(kernel matrix) [cvXP_train] instead of 
                # the train set [cvX_train] in the fit method 
                kernel_type = 'precomputed' 
                                              
            if kFunction == "poly_Features":   # Polynomial features
                
                # Generate a new feature matrix 
                poly = PolynomialFeatures(Parameter)
                cvXK_train = poly.fit_transform(cvX_train)
                cvXK_test = poly.transform(cvX_test)
                
                # fit model using the transformed feature space
                kernel_type = 'linear' 
 
            clf = svm.SVC(random_state=0, kernel=kernel_type).fit(cvXK_train,cvy_train) #learn model
            cvy_test_pred = clf.predict(cvXK_test) #apply model on validation set

            # compute the accuracy rate corresponding to model learnt using pi(Parameter)
            parameters_scores[Parameter] = accuracy_score(cvy_test, cvy_test_pred)

        folds_scores[foldi] = parameters_scores    # Accuracy for every choice of hyper-parameter value per fold 
        
    # Compute the Validation error rate(Average accuracy rate)
    df_params = pd.DataFrame.from_dict(folds_scores)
    series_params = df_params.sum(axis = 1)/n_splits
    
    # Select the best hyper-parameter value
    best_param = series_params.idxmax()
     
    # Learn final model on the train dataset
    if kFunction == "rbf":
        XK_train = rbf_kernel(X_train, gamma=best_param)
    
    if kFunction == "poly_Kernel":
        
        gamma = best_param[0]
        degree = best_param[1]       
        XK_train = polynomial_kernel(X_train, gamma=gamma, degree=degree) 

    if kFunction == "poly_Features":
        # Generate a new feature matrix 
        poly = PolynomialFeatures(best_param)
        XK_train = poly.fit_transform(X_train)
        
    clf = svm.SVC(random_state=0, kernel=kernel_type).fit(XK_train,y_train)
    return(clf, best_param)
    

In [133]:
# Nested Cross Validation
import itertools
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, random_state=123, shuffle=True) #Stratified Cross Validation 
outerfolds_scores = dict()
folds_hyperparameters = dict()

innerCV_splits = 5

cvFolds = list(skf.split(X, Y))

for count, fold in enumerate(cvFolds):

    train_index = fold[0]
    test_index = fold[1]
    
    cvX_train, cvX_test = X[train_index], X[test_index]
    cvy_train, cvy_test = Y[train_index], Y[test_index]
    
    foldi = "outer-fold"+str(count+1)
    
    #scale the data set (both train and test)
    scaler = StandardScaler()
    cvX_train = scaler.fit_transform(cvX_train) 
    cvX_test = scaler.transform(cvX_test)
    
    kalgo_scores = {}    # holds accuracy rate for each space transformation algorithm
    kpi = {}  # Best hyper-parameter for each space transformation algorithm
    
    for key in param_grid.keys():

        if key in ["rbf", "poly_Kernel"]:
            gamma_range = param_grid[key]["gamma"]

        if key in ["poly_Features", "poly_Kernel"]:
            degree_range = param_grid[key]["degree"]
            
        if key == "rbf":    # rbf_kernel
            
            param_sets = gamma_range
            
        if key == "poly_Kernel":    # Polynomial Kernel

            # Permutation of the gamma_range and degree_range
            permu = [list(zip(x,degree_range)) for x in itertools.permutations(gamma_range,len(degree_range))]
            # flatten the list of lists and then get only unique combinations
            param_sets = set(list(itertools.chain(*permu)))
            
        if key == "poly_Features":   # Polynomial features
            param_sets = degree_range
            
        # Inner cross-validation to select the best hyper-parameter value and model learned over cvX_train data set
        model_pi = model_selection(cvX_train, cvy_train, param_sets, innerCV_splits, key)
        
        # Compute accuracy rate  using test set
        model = model_pi[0]
        pi = model_pi[1]
        
        if key == "rbf": 
            # Get the (M x N) Gram matrix(Kernel matrix) data points in the validation/train data set 
            # for best hyper-parameter pi
            cvXK_test = rbf_kernel(cvX_test,cvX_train, gamma=pi)          
        if key == "poly_Kernel":
            gamma = pi[0]
            degree = pi[1] 
            # Get the (M x N) Gram matrix(Kernel matrix) data points in the validation/train data set 
            # for best hyper-parameter pi
            cvXK_test = polynomial_kernel(cvX_test,cvX_train, gamma=gamma, degree=degree)
        if key == "poly_Features":
            # Get new feature space for best hyper-parameter pi
            poly = PolynomialFeatures(pi)
            cvXK_test = poly.fit_transform(cvX_test)
            
        cvy_test_pred = model.predict(cvXK_test)
        kalgo_scores[key] = accuracy_score(cvy_test, cvy_test_pred)
        kpi[key] = pi   # holds best hyperparameters for each space transformation (rbf,poly_Features,poly_Kernel)
    
    outerfolds_scores[foldi] = kalgo_scores    # Accuracy rate per fold for each key(algorithm)
    folds_hyperparameters[foldi] = kpi
    
#Accuracy rate across each outer fold for [rbf/poly_Kernel/poly_Features]
df_outerFolds = pd.DataFrame.from_dict(outerfolds_scores)
df_outerFolds

outer-fold1  outer-fold2  outer-fold3  outer-fold4  outer-fold5
poly_Features     1.000000     0.933333     0.966667          1.0     0.933333
poly_Kernel       1.000000     0.900000     1.000000          1.0     0.900000
rbf               0.966667     0.900000     0.966667          1.0     0.933333

##### Best hyper-parameters for each space transformation (rbf,poly_Features,poly_Kernel) across the outer folds

In [136]:
df_hyperpar = pd.DataFrame.from_dict(folds_hyperparameters)
df_hyperpar

outer-fold1 outer-fold2 outer-fold3 outer-fold4 outer-fold5
poly_Features           2           2           2           2           2
poly_Kernel      (0.5, 2)    (0.1, 5)    (0.5, 2)    (1.0, 2)    (0.5, 2)
rbf                   0.5         0.5         0.5         0.5         0.5

##### Select the algorithm that has the maximum Average Accuracy score

In [127]:
Ave_Scores = df_outerFolds.sum(axis = 1)/n_splits
Ave_Scores

poly_Features    0.966667
poly_Kernel      0.960000
rbf              0.953333
dtype: float64

In [129]:
Ave_Scores.idxmax()

'poly_Features'

## Question 2 (10 pts)
Write the explicit constraints (without using any vectorial notation, as a summation of single variables multiplied by a constant + bias term) of the Support Vector Machine to classify correctly iris dataset (Iris-Versicolor vs. others). In particular use 5 points in Iris-Versicolor, 2 points for iris-setosa, and 3 points for iris Virginia.
Please show the points you selected and after the constraints

${\displaystyle {\vec {w}}\cdot {\vec {x}}_{i}-b\geq 1}$, if ${\displaystyle y_{i}=1}$(class = Iris-versicolor)<br> or <br>
${\displaystyle {\vec {w}}\cdot {\vec {x}}_{i}-b\leq -1}$, if ${\displaystyle y_{i}=-1}$(class = ~Iris-versicolor)

In [130]:
Iris_versicolor = data_iris[data_iris['class'] == "Iris-versicolor"]
nrows = Iris_versicolor.shape[0]
idx = np.random.randint(nrows, size=5)
Iris_versicolor.iloc[idx, :]

sepal-length  sepal-width  petal-length  petal-width            class
84           5.4          3.0           4.5          1.5  Iris-versicolor
59           5.2          2.7           3.9          1.4  Iris-versicolor
51           6.4          3.2           4.5          1.5  Iris-versicolor
74           6.4          2.9           4.3          1.3  Iris-versicolor
77           6.7          3.0           5.0          1.7  Iris-versicolor

$5.4w_{1} + 3.0w_{2} + 4.5w_{3} + 1.5w_{4}-b\geq 1$<br>
$5.2w_{1} + 2.7w_{2} + 3.9w_{3} + 1.4w_{4}-b\geq 1$<br>
$6.4w_{1} + 3.2w_{2} + 4.5w_{3} + 1.5w_{4}-b\geq 1$<br>
$6.4w_{1} + 2.9w_{2} + 4.3w_{3} + 1.3w_{4}-b\geq 1$<br>
$6.7w_{1} + 3.0w_{2} + 5.0w_{3} + 1.7w_{4}-b\geq 1$

In [64]:
Iris_setosa = data_iris[data_iris['class'] == "Iris-setosa"]
nrows = Iris_setosa.shape[0]
idx = np.random.randint(nrows, size=2)
Iris_setosa.iloc[idx, :]

sepal-length  sepal-width  petal-length  petal-width        class
41           4.5          2.3           1.3          0.3  Iris-setosa
19           5.1          3.8           1.5          0.3  Iris-setosa

$4.5w_{1} + 2.3w_{2} + 1.3w_{3} + 0.3w_{4}-b \leq -1$<br>
$5.1w_{1} + 3.8w_{2} + 1.5w_{3} + 0.3w_{4}-b \leq -1$<br>

In [93]:
Iris_virginica = data_iris[data_iris['class'] == "Iris-virginica"]
nrows = Iris_virginica.shape[0]
idx = np.random.randint(nrows, size=3)
Iris_virginica.iloc[idx, :]

sepal-length  sepal-width  petal-length  petal-width           class
105           7.6          3.0           6.6          2.1  Iris-virginica
142           5.8          2.7           5.1          1.9  Iris-virginica
105           7.6          3.0           6.6          2.1  Iris-virginica

$7.6w_{1} + 3.0w_{2} + 6.6w_{3} + 2.1w_{4}-b \leq -1$<br>
$5.8w_{1} + 2.7w_{2} + 5.1w_{3} + 1.9w_{4}-b \leq -1$<br>
$7.6w_{1} + 3.0w_{2} + 6.6w_{3} + 2.1w_{4}-b \leq -1$

# Extra Points ( 5pts)
An unbalanced dataset (e.g. 95% vs 5%) can be problematic even in the training phase. The learned function can be trivial, e.g. always predicting one class.
A possible solution can have a weight for each point in the way that making a mistake in the minority class will coun more w.r.t. the other. Please redefine the likelihood of the logistic regression to consider these weights for each point. Please compute the log-likelihood and its derivatives.
In addition, add to the negative log-likelihood the norm of W (sum of the square of each component) and compute the derivatives. 

#### Re-balancing (weighting) of the logistic loss (negative log likelihood)

$$\text{logLoss (negative log-likelihood) = logL($\beta$)} = -\sum_{i=1}^{n}[y_{i}log(p_{i}) + (1-y_{i})log(1-p_{i})]$$<br>
Our event of interest (probability of success is the minority observations which is related to the first component of the negative log-likelihood: Label = 1. <br> While the second component represents the loss related to the majority observation Label = 0<br><br>
Probability of success $p_{i} = \dfrac{1}{1 + e^{-x_{i}\beta}}$<br> Probability of failure $(1-p_{i}) = \dfrac{1}{1 + e^{x_{i}\beta}}$

The weighted likelihood:<br><br>$L_{w}(\beta) = \displaystyle \prod_{i=1}^{n} (p_{i})^{w_{1}y_{i}}(1-p_{i})^{(1-w_{1})(1-y_{i})}$<br><br>where $w_{1} = \dfrac{n_{1}}{N}$, the ratio of number of observations related to the majority label over the total number of observations<br><br>
$logL_{w}(\beta) =\ell_{w} = \sum_{i=1}^{n}[w_{1}y_{i}log(p_{i}) + (1-w_{1})(1-y_{i})log(1-p_{i})]$<br><br>For weighted logistic regression, the gradient and Hessian are obtained by differentiating the regularized weighted log-likelihood with respect to $\beta$<br><br>

$$\text{regularized weighted log-likelihood = }logL_{w}(\beta) = \sum_{i=1}^{n}w_{i}ln\dfrac{e^{y_{i}x_{i}\beta}}{1+e^{x_{i}\beta}}-\dfrac{\lambda}{2}\left\lVert \beta \right\rVert^{2}$$<br>$$\text{where} \dfrac{\lambda}{2}\left\lVert \beta \right\rVert^{2} \text{is the regularization (penalty) term added to obtain better generalization.}$$
<br><br>In matrix form, the gradient is:<br>$\nabla_{\beta}lnL_{w} = X^{T}W(y-p)-\lambda\beta$<br>where $W = diag(w_{i})$ and $p$ is the probability vector $\dfrac{1}{1 + e^{-x_{i}\beta}}$. <br><br>The Hessian with respect to $\beta$ is then <br>$\nabla_{\beta}^{2}lnL_{w} = -X^{T}DX-\lambda I$<br>where $D =diag(v_{i}w_{i})$ and $v_{i}$ is the variance of $\beta$ 
<br><br>The Newton–Raphson update with respect to $\beta$ on the $(c + 1)$th iteration is<br>$\hat{\beta}^{(c+1)} = (X^{T}DX + \lambda I)^{-1}X^{T}Dz^{c}$<br>where $z^{c}=\hat{\beta}^{c}+ D^{-1}(y-p)$ is the adjusted dependent variable or the adjusted response.<br><br>The weighted least squares (WLS) subproblem is then <br>$(X^{T}DX + \lambda I)\hat{\beta}^{(c+1)}=X^{T}Dz^{c}$
